In [1]:
import numpy as np
import time

In [2]:
#打开词典文件，返回列表
def open_dict(Dict = 'hahah', path=r'/Users/Wan_Ning/Desktop/AmzPyNL/dic/'):
    path = path + '%s.txt' % Dict
    dictionary = open(path, 'r', encoding='utf-8')
    dict = []
    for word in dictionary:
        word = word.strip('\n')
        dict.append(word)
    return dict

def judgeodd(num):
    if (num % 2) == 0:
        return 'even'
    else:
        return 'odd'

In [3]:
#注意，这里你要修改path路径。
deny_word = open_dict(Dict = 'deny_n', path= r'/Users/Wan_Ning/Desktop/AmzPyNL/dic/')
posdict = open_dict(Dict = 'PP', path= r'/Users/Wan_Ning/Desktop/AmzPyNL/dic/')
negdict = open_dict(Dict = 'NN', path= r'/Users/Wan_Ning/Desktop/AmzPyNL/dic/')

degree_word = open_dict(Dict = 'extent', path= r'/Users/Wan_Ning/Desktop/AmzPyNL/dic/')
mostdict = degree_word[degree_word.index('|extreme')+1 : degree_word.index('|very')]#权重4，即在情感词前乘以3
verydict = degree_word[degree_word.index('|very')+1 : degree_word.index('|more')]#权重3
moredict = degree_word[degree_word.index('|more')+1 : degree_word.index('|ish')]#权重2
ishdict = degree_word[degree_word.index('|ish')+1 : degree_word.index('|last')]#权重0.5

In [32]:
def sentiment_score_list(dataset):
    seg_sentence = dataset.split('#')

    count1 = []
    count2 = []
    for sen in seg_sentence: #循环遍历每一个评论
        i = 0 #记录扫描到的词的位置
        a = 0 #记录情感词的位置
        poscount = 0 #积极词的第一次分值
        poscount2 = 0 #积极词反转后的分值
        poscount3 = 0 #积极词的最后分值（包括叹号的分值）
        negcount = 0
        negcount2 = 0
        negcount3 = 0
        for word in sen:
            if word in posdict:  # 判断词语是否是情感词
                poscount += 1
                c = 0
                for w in segtmp[a:i]:  # 扫描情感词前的程度词
                    if w in mostdict:
                        poscount *= 4.0
                    elif w in verydict:
                        poscount *= 3.0
                    elif w in moredict:
                        poscount *= 2.0
                    elif w in ishdict:
                        poscount *= 0.5
                    elif w in deny_word:
                        c += 1
                if judgeodd(c) == 'odd':  # 扫描情感词前的否定词数
                    poscount *= -1.0
                    poscount2 += poscount
                    poscount = 0
                    poscount3 = poscount + poscount2 + poscount3
                    poscount2 = 0
                else:
                    poscount3 = poscount + poscount2 + poscount3
                    poscount = 0
                a = i + 1  # 情感词的位置变化
            elif word in negdict:  # 消极情感的分析，与上面一致
                negcount += 1
                d = 0
                for w in segtmp[a:i]:
                    if w in mostdict:
                        negcount *= 4.0
                    elif w in verydict:
                        negcount *= 3.0
                    elif w in moredict:
                        negcount *= 2.0
                    elif w in ishdict:
                        negcount *= 0.5
                    elif w in degree_word:
                        d += 1
                if judgeodd(d) == 'odd':
                    negcount *= -1.0
                    negcount2 += negcount
                    negcount = 0
                    negcount3 = negcount + negcount2 + negcount3
                    negcount2 = 0
                else:
                    negcount3 = negcount + negcount2 + negcount3
                    negcount = 0
                    break
                a = i + 1
#                     break
#             elif word == '！' or word == '!':  ##判断句子是否有感叹号
#                 for w2 in segtmp[::-1]:  # 扫描感叹号前的情感词，发现后权值+2，然后退出循环
#                     if w2 in posdict or negdict:
#                         poscount3 += 2
#                         negcount3 += 2
                    
            i += 1 # 扫描词位置前移
            # 以下是防止出现负数的情况
            pos_count = 0
            neg_count = 0
            if poscount3 < 0 and negcount3 > 0:
                neg_count += negcount3 - poscount3
                pos_count = 0
            elif negcount3 < 0 and poscount3 > 0:
                pos_count = poscount3 - negcount3
                neg_count = 0
            elif poscount3 < 0 and negcount3 < 0:
                neg_count = -poscount3
                pos_count = -negcount3
            else:
                pos_count = poscount3
                neg_count = negcount3

            count1.append([pos_count, neg_count])
        count2.append(count1)
        count1 = []

    return count2


In [33]:
def sentiment_score(senti_score_list):
    score = []
    for review in senti_score_list:
        score_array = np.array(review)
        Pos = np.sum(score_array[:, 0])
        Neg = np.sum(score_array[:, 1])
        AvgPos = np.mean(score_array[:, 0])
        AvgPos = float('%.1f'%AvgPos)
        AvgNeg = np.mean(score_array[:, 1])
        AvgNeg = float('%.1f'%AvgNeg)
        StdPos = np.std(score_array[:, 0])
        StdPos = float('%.1f'%StdPos)
        StdNeg = np.std(score_array[:, 1])
        StdNeg = float('%.1f'%StdNeg)
        score.append([Pos, Neg, AvgPos, AvgNeg, StdPos, StdNeg])
    return score

In [34]:
review = open('../review.txt','r').read()

print(review)

's , like , littl , just , realli , make , n't , help , thing , use , think , onli , come , need , easi , look , cute , becaus , price , good , doe , ani , want , small , way , month , togeth , love , old , lot#																
kid , use , color , great , make , love , fun , differ , like , easi , way , need , realli , just , want , recommend , becaus , ani , come , set , help , work , purchas , 's , think , buy , price , box , veri , small#								
								
old , year , year old , love , month , bought , play , great , daughter , son , got , easi , togeth , fun , christma , small , buy , enjoy , just , toy , recommend , veri , kid , price , size , qualiti , thing , make , want , come#								
								
toy , good , great , perfect , love , price , size , qualiti , son , play , littl , small , kid , like , just , fun , buy , realli , lot , make , recommend , want , work , look , onli , ani , becaus , need , easi , daughter#								
								
love , gift , daughter , christma , great , bo

In [36]:
score_list = sentiment_score(sentiment_score_list(review))
#     score = score_list[0][2]-score_list[0][3]
score_list

[[0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0],
 [0, 0, 0.0, 0.0, 0.0, 0.0]]